In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient


import numpy as np
import pandas as pd
import scipy.stats as stats

import datetime

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

In [2]:
client = MongoClient()
db_aud = client.aud_scraper

collection_aud = db_aud.rates

In [3]:
df_aud = pd.DataFrame(list(collection_aud.find()))

In [4]:
df_aud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id            344 non-null    object
 1   date           341 non-null    object
 2   change%        343 non-null    object
 3   AUD_rate_tgt%  340 non-null    object
dtypes: object(4)
memory usage: 10.9+ KB


In [5]:
# remove Mongo _id column

df_aud.drop(columns = ['_id'], inplace = True)

In [6]:
# drop rows with NaN values

df_aud.dropna(inplace = True)

In [7]:
df_aud.head()

,date,change%,AUD_rate_tgt%
0,6 May 2020,0.00,0.25
1,8 Apr 2020,0.00,0.25
2,20 Mar 2020,-0.25,0.25
3,4 Mar 2020,-0.25,0.50
4,5 Feb 2020,0.00,0.75


In [8]:
def date_converter(date):
    
    """
    PARAMETERS
    date - string in format '2-digit-day month_name 4-digit-year'
    
    RETURNS
    date in new string format ' 4 digit year-2 digit month-2 digit day'
    """
    
    dct_month = {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }
    
    lst_date = date.split()
        
    str_yr = lst_date[-1]
    
    str_mo = dct_month[lst_date[1]]
    
    str_d = lst_date[0]
    
    #add zeros before single digit dates
    if len(str_d) ==1:
        str_d += '0'
        str_d = str_d[::-1]
        
    
    str_date = f'{str_yr}-{str_mo}-{str_d}'

    return str_date

In [9]:
df_aud['date'] = [date_converter(d) for d in df_aud['date']]

In [10]:
df_aud['date'] = pd.to_datetime(df_aud['date'], yearfirst = True)

In [11]:
df_aud.head()

,date,change%,AUD_rate_tgt%
0,2020-05-06,0.00,0.25
1,2020-04-08,0.00,0.25
2,2020-03-20,-0.25,0.25
3,2020-03-04,-0.25,0.50
4,2020-02-05,0.00,0.75


In [12]:
df_aud['AUD_rate_tgt%'] = [r.split()[-1] for r in df_aud['AUD_rate_tgt%']]
df_aud['change'] = [r.split()[-1] for r in df_aud['change']]


KeyError: 'change'

In [ ]:
df_aud.head()

In [ ]:
df_aud.rename(columns = {'change': 'aud_rate_change'}, inplace = True)

In [ ]:
df_aud['AUD_rate_tgt%'] = pd.to_numeric(df_aud['AUD_rate_tgt%'])

In [ ]:
df_aud.info()

In [ ]:
%store df_aud